In [ ]:
# installing required libraries
! pip install pydub
! pip install static_ffmpeg
! pip install pyaudio
! pip install librosa
! pip install torch
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: Could not build wheels for pyaudio which use PEP 517 and cannot be installed directly
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.3 MB/s 
     |█

In [7]:
# since the model take only wav file as input , so we'll convert the audio format to wav using pydub

from pydub import AudioSegment
from pydub.playback import play
import static_ffmpeg

static_ffmpeg.add_paths()


audio = AudioSegment.from_file(r'audio/test.mp3')

# if you want to convert the audio as whole uncomment the code below
# audio.export("audio export path here", format="wav")

# we'll split the audio in 3 minutes each to minimize the computational power used by model
audio_length = len(audio) / (60 * 1000)

split_marker = 180
split_audio = [audio[:180 * 1000]]

for i in range(round(audio_length / (180 * 1000))):
  split_audio.append(audio[split_marker * 1000:(split_marker + 180) * 1000])
  split_marker += 180

#it will create the file in audio dir, make sure to create audio dir in root folder

count = 0
for count, audio_sample in enumerate(split_audio):
  count += 1
  with open(f'audio/{count}_audi_file.wav', 'wb') as out_f:
    audio_sample.export(out_f, format='wav')

In [ ]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import os

# loading model and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained('facebook/wav2vec2-base-960h')
model = Wav2Vec2ForCTC.from_pretrained('facebook/wav2vec2-base-960h')

print('model loaded working on audio with length :', f'{"{0:.2f}".format(audio_length)}s')

text_arr = []
for i in range(len(split_audio)):
  # loading audio in model from the audio dir
  speech, rate = librosa.load(f'audio/{i+1}_audi_file.wav', sr=16000) 
  
  input_values = tokenizer(speech, return_tensors='pt').input_values
  with torch.no_grad():
      logits = model(input_values).logits
      
  predicted_ids = torch.argmax(logits, dim=-1)
  transcription = tokenizer.batch_decode(predicted_ids)[0]
  text_arr.append(transcription)
  
final_speech = ''
for speech in text_arr:
  final_speech += speech
print(final_speech)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model loaded working on audio with length : 0.36s
